# Cómputo del Modelo Global

---

### Importación de librerías

In [1]:
import os
import numpy as np
import tensorflow as tf
from TheModel import build
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

### Importación de los modelos

In [2]:
# Validar que todos los modelos tienen la misma arquitectura
# Nombres de archivos de modelos
model_paths = [f"lmodel_{i}.keras" for i in range(5)]

# Cargar modelos en una lista
loaded_local_models = []
for path in model_paths:
    try:
        model = tf.keras.models.load_model(path)
        loaded_local_models.append(model)
        print(f"Modelo cargado: {path}")
    except Exception as e:
        print(f"Error al cargar {path}: {e}")

for i in range(len(loaded_local_models) - 1):
    assert loaded_local_models[i].summary() == loaded_local_models[i + 1].summary(), "Models have different architectures"

Modelo cargado: lmodel_0.keras
Modelo cargado: lmodel_1.keras
Modelo cargado: lmodel_2.keras
Modelo cargado: lmodel_3.keras
Modelo cargado: lmodel_4.keras
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 512)               401920    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_10 (Dense)            (N

### 1. FedAvg

In [3]:
local_weights = [model.get_weights() for model in loaded_local_models]
averaged_weights = [np.mean(np.array(w), axis = 0) for w in zip(*local_weights)]

global_model = build.create_model()
global_model.set_weights(averaged_weights)

# Datos de prueba 
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

y_pred = global_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis = 1)

print("=================== FEDAVG ========================")
print(classification_report(y_test, y_pred_classes))

global_model.save("global_model_fedavg.keras")

313/313 [==============================] - 1s 1ms/step
=================== FEDAVG ========================
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.00      0.00      0.00      1135
           2       0.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.10      1.00      0.18       974
           9       0.00      0.00      0.00      1009

    accuracy                           0.10     10000
   macro avg       0.01      0.10      0.02     10000
weighted avg       0.01      0.10      0.02     10000



c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2. FedMedian
FedMedian es un método de agregación robusto utilizado en Aprendizaje Federado. En lugar de promediar directamente los pesos como en FedAvg, FedMedian calcula la mediana elemento por elemento entre los modelos locales. Este enfoque es resistente a valores extremos o outliers que podrían perjudicar la calidad del modelo global, especialmente en entornos no confiables o con datos no independientes e idénticamente distribuidos (non-IID).

In [4]:
local_weights = [model.get_weights() for model in loaded_local_models]
median_weights = [np.median(np.array(weights), axis = 0) for weights in zip(*local_weights)]

global_model_median = build.create_model()
global_model_median.set_weights(median_weights)

# Datos de prueba 
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

y_pred = global_model_median.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis = 1)

print("=================== FEDMEDIAN ========================")
print(classification_report(y_test, y_pred_classes))

global_model_median.save("global_model_fedmedian.keras")

313/313 [==============================] - 0s 1ms/step
=================== FEDMEDIAN ========================
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.67      0.67      0.67      1135
           2       0.78      0.05      0.09      1032
           3       0.43      0.11      0.17      1010
           4       0.59      0.63      0.61       982
           5       0.43      0.26      0.33       892
           6       0.00      0.00      0.00       958
           7       0.83      0.01      0.03      1028
           8       0.20      0.93      0.32       974
           9       0.27      0.61      0.37      1009

    accuracy                           0.33     10000
   macro avg       0.42      0.33      0.26     10000
weighted avg       0.43      0.33      0.26     10000



c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 2. FedTrimmedMean
FedTrimmedMean es otro enfoque robusto para la agregación de modelos en aprendizaje federado. Ordena los valores de cada peso, elimina un porcentaje superior e inferior (por ejemplo, el 20% más alto y más bajo) y luego calcula el promedio del resto. Esta técnica reduce la influencia de valores extremos o posibles ataques de clientes maliciosos. Es útil en escenarios donde la calidad de los clientes varía ampliamente.

In [5]:

def trimmed_mean(values, trim_ratio=0.2):
    """Calcula el trimmed mean para una lista de arrays."""
    values = np.array(values)
    n = len(values)
    trim_n = int(n * trim_ratio)
    sorted_values = np.sort(values, axis=0)
    trimmed_values = sorted_values[trim_n : n - trim_n]
    return np.mean(trimmed_values, axis=0)

# Aplicar FedTrimmedMean capa por capa
trimmed_weights = [trimmed_mean(layer_weights, trim_ratio=0.2) for layer_weights in zip(*local_weights)]

# Crear el modelo global y asignar pesos
global_model_trimmed = build.create_model()
global_model_trimmed.set_weights(trimmed_weights)

# Evaluación
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

y_pred = global_model_trimmed.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("=================== FEDTRIMMEDMEAN ========================")
print(classification_report(y_test, y_pred_classes))

global_model_trimmed.save("global_model_fedtrimmedmean.keras")


313/313 [==============================] - 1s 1ms/step
=================== FEDTRIMMEDMEAN ========================
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       0.55      0.02      0.03      1135
           2       0.61      0.07      0.12      1032
           3       1.00      0.00      0.00      1010
           4       0.72      0.07      0.13       982
           5       0.32      0.03      0.06       892
           6       0.00      0.00      0.00       958
           7       0.00      0.00      0.00      1028
           8       0.12      0.97      0.21       974
           9       0.23      0.34      0.27      1009

    accuracy                           0.15     10000
   macro avg       0.35      0.15      0.08     10000
weighted avg       0.36      0.15      0.08     10000



c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danyj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
